In [1]:
import pandas as pd
df_sample = pd.read_csv("/Users/htg/Desktop/Sydney_University/冒犯性检测/dataset/data_emnlp.csv", sep=",")

In [ ]:
#Qwen2.5 32B CPP 
import requests
import pandas as pd
import time
import re  

url = "https://api.siliconflow.cn/v1/chat/completions"

# 读取数据集
texts = df_sample.iloc[:, 0]

headers = {
    "Authorization": "Bearer Your own key",  
    "Content-Type": "application/json"
}

pinyin = []

# 逐条处理文本
for idx, text in enumerate(texts):
    payload = {
        "model": "Qwen/Qwen2.5-32B-Instruct",
        "messages": [
            {
                "role": "user",
                "content": 
                          f"""
                            "将所有文本按照中文发音转换为不带声调的汉语拼音。\n"
                            "请只输出不带声调拼音,不要输出其他内容，输出内容不要换行\n"

                            文本如下：
                            {text} 
                            """
                            }
        ],
        "stream": False,
        "max_tokens": 1000,
        "stop": None,
        "temperature": 0.1,
        "top_p": 0.9,
        "top_k": 5,
        "frequency_penalty": 0.5,
        "n": 1,
        "response_format": {"type": "text"},
       
    }

    response = requests.request("POST", url, json=payload, headers=headers)

    # 检查 API 响应
    if response.status_code == 200:
        json_response = response.json()
        if "choices" in json_response and len(json_response["choices"]) > 0:
            answer = json_response["choices"][0]["message"]["content"]
            pinyin.append(answer)
            print(f"第 {idx + 1} 条数据拼音：{answer}")



In [ ]:
#Qwen2.5 32B CPP
import requests
import pandas as pd
import time
import re  

url = "https://api.siliconflow.cn/v1/chat/completions"

headers = {
    "Authorization": "Bearer Your own key",  
    "Content-Type": "application/json"
}

df = pd.DataFrame({
    "text": texts,
    "pinyin": pinyin
})

# 合并成一个新列
df["combined"] = df["text"] + " | " + df["pinyin"]
pin_text = df["combined"]

results = []

# 逐条处理文本
for idx, text in enumerate(pin_text):
    payload = {
        "model": "Qwen/Qwen2.5-32B-Instruct",
        "messages": [
            {
                "role": "user",
                "content": 
                          f"""
                            你是一个中文冒犯性语言检测专家
                            请结合汉语拼音和文本判断是否具有冒犯性
                            这里的冒犯性主要指包含人身攻击、侮辱、歧视、仇恨言论或极端粗俗的内容。
                            输出格式:
                            结论: <0或1>
                            文本如下：
                            {text} 
                            """
                            }
        ],
        "stream": False,
        "max_tokens": 1000,
        "stop": None,
        "temperature": 0.1,
        "top_p": 0.9,
        "top_k": 5,
        "frequency_penalty": 0.5,
        "n": 1,
        "response_format": {"type": "text"},
       
    }

    response = requests.request("POST", url, json=payload, headers=headers)

    # 检查 API 响应
    if response.status_code == 200:
        json_response = response.json()
        if "choices" in json_response and len(json_response["choices"]) > 0:
            answer = json_response["choices"][0]["message"]["content"]
            match = re.search(r"结论\s*[:：]\s*([01])", answer)
            label = match.group(1) if match else ""
            results.append(label)
            print(f"第 {idx + 1} 条数据判断结果：{label}")
    
    time.sleep(2)  

# 保存结果到 CSV 文件
result_df = pd.DataFrame({"Content": texts, "Result": results})

result_df.to_csv("./Pinyin_result/Qwen2.5_32B_results.csv", sep="\t", index=False)